In [7]:
import os
import pandas as pd
from collections import Counter
from kiwipiepy import Kiwi
from keybert import KeyBERT
from transformers import BertModel

# 데이터셋 경로 설정
data_folder = "Completed_csv"

# 형태소 분석기, KeyBERT 불러오기
kiwi = Kiwi()
bert_model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(bert_model)

# 불용어 리스트
custom_stopwords = [
    "지난해", "이날", "오늘", "내일", "올해", "시간", "관련", 
    "있다", "없다", "위해", "대한", "그리고", "등", "수", "것"
]

# 명사 추출
def extract_nouns(text):
    results = []
    analysis = kiwi.analyze(text)
    for token, pos, _, _ in analysis[0][0]:
        if len(token) > 1 and (pos.startswith('N') or pos.startswith('SL')) and token not in custom_stopwords:
            results.append(token)
    return results

# CSV 파일에서 데이터 로드 => title + short_content 텍스트 결합
def load_and_combine_text(data_folder):
    combined_texts = []

    for filename in os.listdir(data_folder):
        if filename.endswith(".csv"):
            filepath = os.path.join(data_folder, filename)
            df = pd.read_csv(filepath, encoding="utf-8")

            # `title`과 `short_content` 열 결합
            if "title" in df.columns and "short_content" in df.columns:
                df["combined"] = df["title"].fillna("") + " " + df["short_content"].fillna("")
                combined_texts.extend(df["combined"].tolist())

    return combined_texts

# 키워드 추출
def extract_top_keywords(texts, top_n=50):
    all_nouns = []

    for text in texts:
        # 명사 추출
        nouns = extract_nouns(text)
        all_nouns.extend(nouns)

    # 키워드 빈도 계산
    word_counts = Counter(all_nouns)
    return word_counts.most_common(top_n)


if __name__ == "__main__":
    # 데이터 로드 및 텍스트 결합
    combined_texts = load_and_combine_text(data_folder)

    # 상위 키워드 추출
    top_keywords = extract_top_keywords(combined_texts, top_n=50)

    # 결과 출력
    print("상위 50개 키워드:")
    for rank, (word, count) in enumerate(top_keywords, start=1):
        print(f"{rank}. {word} - {count}회 등장")


상위 50개 키워드:
1. 대통령 - 520회 등장
2. 체포 - 254회 등장
3. 윤석열 - 227회 등장
4. 공수처 - 150회 등장
5. 조사 - 147회 등장
6. 정부 - 146회 등장
7. 집행 - 130회 등장
8. 서울 - 128회 등장
9. 미국 - 125회 등장
10. 내란 - 122회 등장
11. 영장 - 110회 등장
12. 국민 - 109회 등장
13. 장관 - 108회 등장
14. 대행 - 98회 등장
15. 국회 - 92회 등장
16. 권한 - 90회 등장
17. 한국 - 90회 등장
18. 관저 - 83회 등장
19. 탄핵 - 80회 등장
20. 비상계엄 - 80회 등장
21. 경찰 - 79회 등장
22. 수사 - 77회 등장
23. 지원 - 77회 등장
24. 계엄 - 76회 등장
25. 경호처 - 74회 등장
26. 혐의 - 73회 등장
27. 최상목 - 71회 등장
28. 트럼프 - 71회 등장
29. 국가 - 70회 등장
30. 시장 - 69회 등장
31. 의원 - 69회 등장
32. 범죄 - 66회 등장
33. 경제 - 64회 등장
34. 사태 - 59회 등장
35. 경기 - 59회 등장
36. 경호 - 57회 등장
37. 기획 - 57회 등장
38. 교육 - 56회 등장
39. 기관 - 55회 등장
40. 일본 - 55회 등장
41. 기업 - 55회 등장
42. 이후 - 54회 등장
43. 고위 - 53회 등장
44. 위원회 - 52회 등장
45. 오후 - 52회 등장
46. 영상 - 52회 등장
47. 시작 - 52회 등장
48. 거부 - 51회 등장
49. 최대 - 51회 등장
50. 재정부 - 50회 등장
